In [ ]:
# alignparse-environment

In [ ]:
# need paths to .fastq and ccs summaries
# reads from ccs_1592 and ccs_1608

In [ ]:
# this is taking a while to import alignparse.targets
import time
import yaml
import numpy as np
import pandas as pd

import alignparse
import alignparse.targets
import alignparse.minimap2
from alignparse.constants import CBPALETTE

from plotnine import *
import math

In [ ]:
# setup the config file
with open('pkr_config.yaml') as f:
    config = yaml.safe_load(f)
config

In [ ]:
# setup alignparse targets
targets = alignparse.targets.Targets(
                seqsfile=config['amplicons'],
                feature_parse_specs=config['feature_parse_specs'])

In [ ]:
# plot targets
fig = targets.plot(ax_width=7,
                   plots_indexing='biopython',  # numbering starts at 0
                   ax_height=2,  # height of each plot
                   hspace=1.2,  # vertical space between plots
                   )

In [ ]:
# check the specs used to parse the target amplicons
print(targets.feature_parse_specs('yaml'))

In [ ]:
# setup pacbio run info

# adding a fastq column in the .csv
pacbio_runs = (
    pd.read_csv(config['pacbio_runs'], dtype=str)
    .drop(columns=['subreads'])
    .assign(name=lambda x: x['library'] + '.' + x['run'])
    )
pacbio_runs

In [ ]:
pacbio_runs['fastq'][0]

In [ ]:
# setup minimap2
# I need to figure out how to specify minimap 2 arguments
# currently calling: '-A2', '-B4', '-O12', '-E2', '--end-bonus=13', '--secondary=no', '--cs'
mapper = alignparse.minimap2.Mapper(alignparse.minimap2.OPTIONS_CODON_DMS)

# add more cpus to minimap2
#mapper.options = mapper.options + [f"-t {config['max_cpus']}"]
mapper.options = mapper.options + [f"-t 10"]
mapper.options

In [ ]:
### THE MEAT
# align and parse pacbio reads to target amplicons

# how long does this take
start = time.time()

readstats_csv, aligned_csv, filtered_csv = targets.align_and_parse(
        df=pacbio_runs,
        mapper=mapper,
        outdir=config['process_ccs_dir'],
        name_col='run',
        group_cols=['name', 'library'],
        queryfile_col='fastq',
        overwrite=True,
        ncpus=config['max_cpus'],
        to_csv=True
        )
print('Time: ', time.time() - start)

In [ ]:
# save readstats_csv
readstats_csv.to_csv(config['process_ccs_dir'] + '/readstats.csv', index=False)

In [ ]:
# plot readstats
start = time.time()
readstats_csv = readstats_csv.assign(
    category_all_targets=lambda x: x['category'].str.split().str[0],
    target=lambda x: x['category'].str.split(None, 1).str[1],
    valid=lambda x: x['category_all_targets'] == 'aligned')

ncol = 7
p = (
    ggplot(readstats_csv
           .groupby(['name', 'category_all_targets', 'valid'])
           .aggregate({'count': 'sum'})
           .reset_index(),
           aes('category_all_targets', 'count', fill='valid')) +
    geom_bar(stat='identity') +
    facet_wrap('~ name', ncol=ncol) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(1.85 * min(ncol, len(pacbio_runs)),
                       2 * math.ceil(len(pacbio_runs) / ncol)),
          panel_grid_major_x=element_blank(),
          legend_position='none',
          ) +
    scale_fill_manual(values=CBPALETTE)
    )

p.save('img/rev_readstats.png')
_ = p.draw()

print('Time:', time.time() - start)

In [ ]:
# create filtered_df
# this should be a dictionary of dfs
filtered = {}
for key, value in filtered_csv.items():
    filtered[key] = pd.read_csv(value)

# reasons reads are being filtered out
other_cutoff = 0.02  # group as "other" reasons with <= this frac

filtered_df = (
    pd.concat(df.assign(target=target) for target, df in filtered.items())
    .groupby(['library', 'name', 'run', 'filter_reason'])
    .size()
    .rename('count')
    .reset_index()
    .assign(tot_reason_frac=lambda x: (x.groupby('filter_reason')['count']
                                       .transform('sum')) / x['count'].sum(),
            filter_reason=lambda x: np.where(x['tot_reason_frac'] > other_cutoff,
                                                x['filter_reason'],
                                                'other')
            )
    )

# plot reasons why reads were filtered from each library
ncol = 7
nreasons = filtered_df['filter_reason'].nunique()

p = (
    ggplot(filtered_df, aes('filter_reason', 'count')) +
    geom_bar(stat='identity') +
    facet_wrap('~ name', ncol=ncol) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.25 * nreasons * min(ncol, len(pacbio_runs)),
                       2 * math.ceil(len(pacbio_runs) / ncol)),
          panel_grid_major_x=element_blank(),
          )
    )
p.save('img/rev_filtered.png')
_ = p.draw()

In [ ]:
# count total filtered variants total
filtered_df['count'].sum()

In [ ]:
# combine runs for each library
p = (
    ggplot(filtered_df
           .groupby(['library', 'filter_reason'])
           .aggregate({'count': 'sum'})
           .reset_index(),
           aes('filter_reason', 'count')) +
    geom_bar(stat='identity') +
    facet_wrap('~ library', nrow=1) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.3 * nreasons * pacbio_runs['library'].nunique(), 2),
          panel_grid_major_x=element_blank(),
          )
    )
_ = p.draw()

In [ ]:
# save aligned reads as processed_ccs file
aligned = {}
for key, value in aligned_csv.items():
    aligned[key] = pd.read_csv(value)

aligned_df = (
    pd.concat(df.assign(target=target) for target, df in aligned.items())
    .drop(columns=['query_clip5', 'query_clip3', 'run','name'])
    .rename(columns={'barcode_sequence': 'barcode'})
    )
# write aligned reads to a processed ccs file
aligned_df.to_csv(config['processed_ccs_file'], index=False)

In [ ]:
# plot the number of reads per target 
p = (
    ggplot(readstats_csv
           .groupby(['target'])
           .aggregate({'count': 'sum'})
           .reset_index(), 
           aes('target', 'count')) +
    geom_point(stat='identity', size=3) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.3 * readstats_csv['target'].nunique(), 2),
          panel_grid_major_x=element_blank(),
          ) +
    scale_y_log10(name='number of reads')
    )
_ = p.draw()

In [ ]:
# plot aligned vs filtered reads per target
p = (
    ggplot(readstats_csv
           .groupby(['target', 'valid'])
           .aggregate({'count': 'sum'})
           .reset_index()
           .assign(total=lambda x: x.groupby('target')['count'].transform('sum'),
                   frac=lambda x: x['count'] / x['total'],
                   ), 
           aes('target', 'frac', fill='valid')) +
    geom_bar(stat='identity') +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.5 * readstats_csv['target'].nunique(), 2),
          panel_grid_major_x=element_blank(),
          ) +
    scale_fill_manual(values=CBPALETTE)
    )
_ = p.draw()

In [ ]:
# create filtered_df
# this should be a dictionary of dfs
filtered = {}
for key, value in filtered_csv.items():
    filtered[key] = pd.read_csv(value)

# reasons reads are being filtered out
other_cutoff = 0.02  # group as "other" reasons with <= this frac

filtered_df = (
    pd.concat(df.assign(target=target) for target, df in filtered.items())
    .groupby(['library', 'name', 'run', 'filter_reason'])
    .size()
    .rename('count')
    .reset_index()
    .assign(tot_reason_frac=lambda x: (x.groupby('filter_reason')['count']
                                       .transform('sum')) / x['count'].sum(),
            filter_reason=lambda x: np.where(x['tot_reason_frac'] > other_cutoff,
                                                x['filter_reason'],
                                                'other')
            )
    )

In [ ]:
# plot reasons why reads were filtered from each library
ncol = 7
nreasons = filtered_df['filter_reason'].nunique()

p = (
    ggplot(filtered_df, aes('filter_reason', 'count')) +
    geom_bar(stat='identity') +
    facet_wrap('~ name', ncol=ncol) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.25 * nreasons * min(ncol, len(pacbio_runs)),
                       2 * math.ceil(len(pacbio_runs) / ncol)),
          panel_grid_major_x=element_blank(),
          )
    )
_ = p.draw()

In [ ]:
# combine runs for each library
p = (
    ggplot(filtered_df
           .groupby(['library', 'filter_reason'])
           .aggregate({'count': 'sum'})
           .reset_index(),
           aes('filter_reason', 'count')) +
    geom_bar(stat='identity') +
    facet_wrap('~ library', nrow=1) +
    theme(axis_text_x=element_text(angle=90),
          figure_size=(0.3 * nreasons * pacbio_runs['library'].nunique(), 2),
          panel_grid_major_x=element_blank(),
          )
    )
_ = p.draw()

In [ ]:
# view aligned reads
aligned = {}
for key, value in aligned_csv.items():
    aligned[key] = pd.read_csv(value)

aligned_df = (
    pd.concat(df.assign(target=target) for target, df in aligned.items())
    .drop(columns=['query_clip5', 'query_clip3', 'run','name'])
    .rename(columns={'barcode_sequence': 'barcode'})
    )
# write aligned reads to a processed ccs file
aligned_df.to_csv(config['processed_ccs_file'], index=False)